In [1]:
import gzip
import numpy as np
from numpy import pi as pi

In [2]:
def GetImageData(f):
    # magic number
    f.read(4) 
    
    # number of images
    num = f.read(4)
    num = int.from_bytes(num, byteorder='big') #60000

    row = f.read(4)
    row = int.from_bytes(row, byteorder='big') #28

    column = f.read(4)
    column = int.from_bytes(column, byteorder='big') #28

    buf = f.read(row * column * num)
    data = np.frombuffer(buf, dtype=np.uint8).astype(np.float32)
    data = data.reshape(num, row * column)
    
    return [data, num, row, column]

In [3]:
def GetLabelData(f):
    # magic number
    f.read(4) 

    # number of items
    num = f.read(4)
    num = int.from_bytes(num, byteorder='big') #60000

    buf = f.read(num)
    data = np.frombuffer(buf, dtype=np.uint8)
    data = data.reshape(num)
    
    return [data, num]

In [4]:
import matplotlib.pyplot as plt

train_image = 'train-images-idx3-ubyte.gz'
train_label = 'train-labels-idx1-ubyte.gz'

test_image = 't10k-images-idx3-ubyte.gz'
test_label = 't10k-labels-idx1-ubyte.gz'

# Train
f = gzip.open(train_image,'rb')
data = GetImageData(f)
train_image_data, train_image_num, row, column = data

f = gzip.open(train_label,'rb')
data = GetLabelData(f)
train_label_data, train_label_num = data

# Test
f = gzip.open(test_image,'rb')
data = GetImageData(f)
test_image_data, test_image_num, row, column = data

f = gzip.open(test_label,'rb')
data = GetLabelData(f)
test_label_data, test_label_num = data

# for i in range(5):
#     image = np.asarray(image_data[i]).reshape(row, column)
#     plt.imshow(image)
#     plt.show()

print(f'Train Image Shape: {train_image_data.shape}')
print(f'Train Label Shape: {train_label_data.shape}')
print(f'Test Image Shape: {test_image_data.shape}')
print(f'Test Label Shape: {test_label_data.shape}')

Train Image Shape: (60000, 784)
Train Label Shape: (60000,)
Test Image Shape: (10000, 784)
Test Label Shape: (10000,)


In [5]:
def GetIndexOfEachLabel(label_data):
    index = 0
    index_m = []
    for num in range(10):
        index_m.append([])
        for label in label_data:
            if label == num:
                index_m[num].append(index)
            index +=1
        index = 0
        
    return index_m

In [6]:
def GetMeanVarOfEachPixel(image_data, index_m):
    tmp = []
    mean_var = []

    for num in range(10):
        for i in index_m[num]:
            tmp.append(image_data[i])
        tmp_mean = np.mean(tmp, axis=0)
        tmp_var = np.var(tmp, axis=0)
        
        tmp_var = np.where(tmp_var==0, 1000, tmp_var)
            
        mean_var.append([tmp_mean, tmp_var])
        tmp = []
    
    return mean_var

In [7]:
def GetGaussianP(x, mean, sigma_2):

    coef = 1 / pow(2 * pi * sigma_2, 0.5)
    normal = -0.5 * (pow(x - mean, 2) / sigma_2)
    
    return np.log(coef) + normal

In [8]:
def GetPosterior(index, label):
    posterior = 0

    prior = np.log(len(train_index[label]) / train_label_num)

    for pixel in range(row * column):
        posterior += GetGaussianP(test_image_data[index][pixel], \
                                   train_mean_var[label][0][pixel], \
                                   train_mean_var[label][1][pixel]) + prior

    return posterior

In [9]:
def GetPredictValue(i):
    posterior_m = []
    posterior_normalize = []

    for label in range(10):
        posterior_m.append(GetPosterior(i, label))

    total = sum(posterior_m)

    for label in range(10):
        normal = posterior_m[label] / total
#         print(f'{label} : {normal}')
        posterior_normalize.append(normal)

    return np.argmin(posterior_normalize)

In [10]:
train_index = GetIndexOfEachLabel(train_label_data)
test_index = GetIndexOfEachLabel(test_label_data)

train_mean_var = GetMeanVarOfEachPixel(train_image_data, train_index)

In [11]:
error = 0
for i in range(10):
    predict_val = GetPredictValue(i)
    if predict_val != test_label_data[i]:
        error += 1
print(f'Error rate : {error / 10}')

Error rate : 0.2


In [12]:
GetPredictValue(0)

7

In [13]:
# train_image_classify = np.trunc(train_image_data / 8)
test_image_classify = np.trunc(test_image_data / 8).astype(int)

In [14]:
len(train_index[0])

5923

In [15]:
def GetProbOfEachBin(label):
    tmp = []
    count = []
    for i in test_index[label]:
        tmp.append(test_image_classify[i])

    tmp = np.array(tmp).transpose()
    
    tmp_c = []
    for b in range(32):
        for pixel in range(784):
            tmp_c.append((tmp[pixel] == b).sum())
        count.append(tmp_c)
        tmp_c = []

    count = np.array(count).transpose()
    count = np.where(count == 0, 0.0001, count)
    prob = count / len(test_index[label])
    
    return prob

In [16]:
a = np.array([1,1,2,3,3,3,4,5])
(a == 3).sum()

3

In [17]:
prob_each_bin = []
for label in range(10):
    prob_each_bin.append(GetProbOfEachBin(label))
np.array(prob_each_bin).shape

(10, 784, 32)

In [18]:
def GetPosteriorDC(index, label):
    posterior = 0

    prior = np.log(len(train_index[label]) / train_label_num)
    
    pixel = 0
    for b in test_image_classify[index]:
        posterior += np.log(prob_each_bin[label][pixel][b]) + prior
        pixel += 1

    return posterior

In [19]:
GetPosteriorDC(2, 0)

-2406.6375296755855

In [20]:
def GetPredictValueDC(i):
    posterior_m = []
    posterior_normalize = []

    for label in range(10):
        posterior_m.append(GetPosteriorDC(i, label))

    total = sum(posterior_m)

    for label in range(10):
        normal = posterior_m[label] / total
#         print(f'{label} : {normal}')
        posterior_normalize.append(normal)
        
    return np.argmin(posterior_normalize)

In [21]:
GetPredictValueDC(0)

7

In [22]:
error = 0
for i in range(100):
    predict_val = GetPredictValueDC(i)
    if predict_val != test_label_data[i]:
        error += 1
print(f'Error rate : {error / 100}')

Error rate : 0.11


In [23]:
def PrintClassifyImage(label):
    image = []
    for pixel in range(784):
        black = np.sum(prob_each_bin[label][pixel][16:32])
        white = np.sum(prob_each_bin[label][pixel][:16])

        if black > white:
            image.append(1)
        else:
            image.append(0)

    image = np.array(image).reshape(28, 28)
    
    for i in range(28):
        for j in range(28):
            print(image[i][j], " ", end = "")
        print('\n')